In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import src

In [50]:
pred1 = pd.read_csv("./predictions/rolling_model.csv")
pred2 = pd.read_csv("./predictions/recursive_model.csv")
del pred1['date']
del pred2['date']

In [91]:
#### pred_df = pd.merge(pred1, pred2, on=['country_name', 'location_key', 'population', 'target_date', 'target'], how="left", suffixes=['_roll', '_recursive'])
pred_df['target_date'] = pd.to_datetime(pred_df['target_date'])
pred_df['day_no'] = (pred_df['target_date'] - pred_df['target_date'].min()).dt.days
weight = {
    0: 0.8,
    1: 0.4,
    2: 0.2,
    3: 0.2,
    4: 0.1,
    5: 0.1,
    6: 0.0,
    7: 0.0
}
pred_df["prediction"] = pred_df.apply(lambda x: weight[x['day_no']] * x['prediction_recursive'] + (1 - weight[x['day_no']]) * x['prediction_roll'], axis=1)

In [95]:
pred_df.to_csv("./predictions/ensemble_model.csv", index=False)

In [92]:
pred_df.head()

,country_name,location_key,target_date,target,prediction_roll,population,prediction_recursive,day_no,prediction
0,Argentina,AR_A,2021-07-01,0.041661,0.045330,1406584.0,0.045346,0,0.045342
1,Argentina,AR_A,2021-07-02,0.043367,0.048766,1406584.0,0.064839,1,0.055195
2,Argentina,AR_A,2021-07-03,0.029717,0.035592,1406584.0,0.052885,2,0.039050
3,Argentina,AR_A,2021-07-04,0.011517,0.017349,1406584.0,0.032149,3,0.020309
4,Argentina,AR_A,2021-07-05,0.041803,0.042937,1406584.0,0.050728,4,0.043716


In [93]:
score = src.util.score(pred_df['population'], pred_df['prediction'], pred_df['target'])

In [94]:
score

0.001921918988424349